In [ ]:
%pip install -q -r requirements.txt

In [3]:
from dotenv import load_dotenv
import os
from groq import Groq


load_dotenv()

True

In [8]:
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

Fast language models, also known as efficient language models or lightweight language models, have gained significant attention in recent years due to their ability to process and generate natural language text at speeds and scales not previously possible. Here are some key reasons why fast language models are important:

1. **Real-time applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and customer service platforms to respond quickly to user queries, improving the overall user experience.
2. **Massive scale**: With the ability to process large volumes of text quickly, fast language models can handle massive amounts of user-generated content, social media posts, and online reviews, enabling applications like sentiment analysis, topic modeling, and language translation.
3. **Low-latency interactions**: Fast language models can facilitate low-latency interactions, such as generating responses to voice commands or text-based inputs, e

In [11]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content


In [18]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()


def calculate(operation: str) -> float:
    return eval(operation)


def get_planet_mass(planet) -> float:
    if planet == "earth":
        return 5.972e24
    if planet == "mars":
        return 6.39e23
    if planet == "jupiter":
        return 1.898e27
    if planet == "saturn":
        return 5.683e26
    if planet == "uranus":
        return 8.681e25
    if planet == "neptune":
        return 1.024e26
    if planet == "mercury":
        return 3.285e23
    if planet == "venus":
        return 4.867e24
    return None

In [19]:
neil_tyson = Agent(client=client, system=system_prompt)

In [20]:
result = neil_tyson("What is the mass of Mercury times 5?")
print(result)

Thought: I need to find the mass of Mercury


In [22]:
result = neil_tyson(result)
print(result)

Action: get_planet_mass: Mercury
PAUSE


In [23]:
result = get_planet_mass("mercury")
print(result)

3.285e+23


In [24]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: 3.285e+23'

In [27]:
result = neil_tyson(next_prompt)
print(result)

Thought: I need to multiply this by 5


In [28]:
result = neil_tyson(next_prompt)
print(result)

Action: calculate: 3.285e+23 * 5
PAUSE


In [29]:
result = calculate("3.285e23 * 5")
print(result)

1.6425e+24


In [30]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: 1.6425e+24'

In [31]:
result = neil_tyson(next_prompt)
result

In [35]:
for msg in neil_tyson.messages:
    print(msg['content'])

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 